In [2]:
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [3]:
portal_client = get_portal_client()

In [ ]:
df = portal_client.query_to_pandas(
    """SELECT
        test.id AS test_id,
        template_task.id AS task_id
    FROM topaz.api_service_tests AS test
    INNER JOIN topaz.api_sites AS site
    ON test.site_id = site.id
    LEFT JOIN topaz.api_toc_rows AS toc
    ON (site.toc_id = toc.toc_id)
    AND (
        (test.domain = toc.domain) OR
        (toc.domain IS NULL)
    )
    AND (
        (test.panel = toc.panel) OR
        (toc.panel IS NULL)
    )
    AND (
        (test.loop = toc.loop) OR
        (toc.loop IS NULL)
    )
    AND (test.address = toc.address)
    AND (test.address_type = toc.address_type)
    AND (
        (
            (
                (test.sub_address = '') OR
                (test.sub_address IS NULL)
            ) AND (
                (toc.sub_address IS NULL) OR
                (toc.sub_address = 0)
            )
        ) OR
        (test.sub_address = toc.sub_address)
    )
    INNER JOIN topaz.api_service_tasks AS template_task
    ON site.template_id = template_task.template_id
    AND test.updated_at BETWEEN template_task.created_at AND IFNULL(template_task.deleted_at, CURRENT_TIMESTAMP())
    AND (
        (toc.lcs_device_type = template_task.asset_type) OR
        (template_task.asset_type = '')
    )
    AND template_task.type IN (1, 3)
    WHERE test.id >= 14029088
    AND NOT EXISTS (
        SELECT 1
        FROM topaz.api_service_tasks_record AS task_record
        WHERE task_record.test_id = test.id
        AND task_record.task_id = template_task.id
    )
    LIMIT 100
    """
)

In [ ]:
df

,test_id,task_id
